In [19]:
import MySQLdb
import pandas as pd
import json
import urllib2
from sklearn import tree
import time
import datetime
def klout_getId(screenname):
    url = 'http://api.klout.com/v2/identity.json/twitter?screenName={0}&key=memp3ncn4qvp6c8guzjcc8dp'.format(screenname)    
    try:
        return json.load(urllib2.urlopen(url))
    except:
        return None
def getDBInstance():
    return MySQLdb.connect(host="uncg.saadmtsa.club",    # your host, usually localhost
                         user="root",         # your username
                         passwd="vJnVubg49U",  # your password
                         db="twitter")
def klout_getId(screen_name):
    url = 'http://api.klout.com/v2/identity.json/twitter?screenName={0}&key=memp3ncn4qvp6c8guzjcc8dp'.format(screen_name)    
    try:
        return json.load(urllib2.urlopen(url))
    except:
        return None
    
def klout_getScore(kloutId):
    url = 'http://api.klout.com/v2/user.json/{0}/score?key=memp3ncn4qvp6c8guzjcc8dp'.format(kloutId)
    try:
        return json.load(urllib2.urlopen(url))
    except:
        return None

In [23]:
myDB = getDBInstance()
users = pd.read_sql("SELECT screen_name, if(followers_count = 0, 1, followers_count) as followers_count, \
                if(friends_count = 0, 1, friends_count) as friends_count,\
                statuses_count, YEAR(created_at) as year_created, if(verified, 1, 0) as verified,\
                klout_score FROM user u",
                    con = myDB, index_col = 'screen_name')
users


,followers_count,friends_count,statuses_count,year_created,verified,klout_score
screen_name,,,,,,
Bryce__Foster,379,411,2332,2012,0,0.0
tmj_FLD_HRTA,338,270,144,2009,0,0.0
tmj_BHM_adm,286,279,9,2009,0,0.0
tmj_AZS_adm,312,263,72,2009,0,0.0
HIKARENLALA,160,11,3825,2012,0,0.0
tmj_TUL_adm,293,279,8,2009,0,0.0
tmj_KYL_retail,340,296,212,2009,0,0.0
tmj_MTP_retail,321,292,18,2009,0,0.0
todd_malone,179,268,662,2012,0,0.0


In [3]:
features = users[["followers_count","friends_count","statuses_count","year_created"]]
labels = users["verified"]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(features, labels)

In [4]:
clf.feature_importances_

array([0.452525  , 0.25067072, 0.22767772, 0.06912656])

In [22]:
klout_scores = []
now = datetime.datetime.now()
i = 0
last_day = "{0}-{1}-{2}".format(now.day, now.month, now.year)
for name in users.index.values:
    current_day = "{0}-{1}-{2}".format(now.day, now.month, now.year)
    if last_day != current_day:
        i = 0
    id = klout_getId(name)
    i += 1
    if id is None:
        klout_scores.append(None)
        continue
    time.sleep(0.02)
    klout_scores.append(klout_getScore(id))
    i += 1
    if i >= 20000:
        while last_day == current_day:
            current_day = "{0}-{1}-{2}".format(now.day, now.month, now.year)

19